<a href="https://colab.research.google.com/github/LaurelleJinelle/Formative-2---Data-Preprocessing_-Group-3/blob/main/Formative_2_Group_3_Data_Preparation_and_Prediction_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Preprocessing, Merge and Prediction Model

##Imports and Data Loading

I began by importing the necessary libraries for data manipulation (Pandas) and machine learning (Scikit-Learn, XGBoost). I proceeded to load two distinct datasets: Social Profiles, which contains customer sentiment and engagement data, and Transactions, which contains purchase history. These are loaded directly from raw CSV URLs."

In [27]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pandas as pd

In [3]:
url_social = "https://raw.githubusercontent.com/LaurelleJinelle/Formative-2---Data-Preprocessing_-Group-3/refs/heads/main/customer_social_profiles%20-%20customer_social_profiles.csv"
url_trans = "https://raw.githubusercontent.com/LaurelleJinelle/Formative-2---Data-Preprocessing_-Group-3/refs/heads/main/customer_transactions%20-%20customer_transactions.csv"

In [4]:
df_social = pd.read_csv(url_social)
df_trans = pd.read_csv(url_trans)

##Data Exploration and Inspection
Preliminary Exploratory Data Analysis is performed to understand the structure of the data. We check data types, missing values, and summary statistics. Crucially, we attempt to find an overlap between the two datasets. We observe that the customer_id formats do not match (A178 vs 178), resulting in an intersection count of 0. This indicates a need for data cleaning before merging.

In [5]:
print("=== SOCIAL PROFILES ===")
print(df_social.head())
print(df_social.info())
print(df_social.describe(include='all'))


=== SOCIAL PROFILES ===
  customer_id_new social_media_platform  engagement_score  \
0            A178              LinkedIn                74   
1            A190               Twitter                82   
2            A150              Facebook                96   
3            A162               Twitter                89   
4            A197               Twitter                92   

   purchase_interest_score review_sentiment  
0                      4.9         Positive  
1                      4.8          Neutral  
2                      1.6         Positive  
3                      2.6         Positive  
4                      2.3          Neutral  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   customer_id_new          155 non-null    object 
 1   social_media_platform    155 non-null    object 
 2   engageme

In [6]:
print("\n=== TRANSACTIONS ===")
print(df_trans.head())
print(df_trans.info())
print(df_trans.describe(include='all'))


=== TRANSACTIONS ===
   customer_id_legacy  transaction_id  purchase_amount purchase_date  \
0                 151            1001              408    2024-01-01   
1                 192            1002              332    2024-01-02   
2                 114            1003              442    2024-01-03   
3                 171            1004              256    2024-01-04   
4                 160            1005               64    2024-01-05   

  product_category  customer_rating  
0           Sports              2.3  
1      Electronics              4.2  
2      Electronics              2.1  
3         Clothing              2.8  
4         Clothing              1.3  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customer_id_legacy  150 non-null    int64  
 1   transaction_id      150 non-null    int64  
 2   purchase_amo

In [8]:
print("Unique in social profiles:", df_social['customer_id_new'].nunique())
print("Unique in transactions:", df_trans['customer_id_legacy'].nunique())

# How many of the transaction customers appear in the social dataset
common = set(df_social['customer_id_new']).intersection(df_trans['customer_id_legacy'])
print("Overlap count:", len(common))


Unique in social profiles: 84
Unique in transactions: 75
Overlap count: 0


##Feauture Engineering
Since a customer can have multiple transactions, I chose not to simply merge the tables one-to-one. I grouped the transaction data by customer_id_legacy to create a single summary row per customer. I calculated:

Total/Average Spent: To gauge customer value.

Transaction Count: To measure frequency.

Recency: I calculated how many days have passed since their last purchase (lower is better).

Last Product Category: I extracted the most recent item bought to serve as our target variable for prediction.

In [12]:
df_trans['purchase_date'] = pd.to_datetime(df_trans['purchase_date'])

In [13]:
agg = df_trans.groupby('customer_id_legacy').agg({
    'purchase_amount': ['count', 'sum', 'mean'],
    'purchase_date': ['max', 'min'],
    'customer_rating': 'mean'
}).reset_index()

agg.columns = [
    'customer_id_legacy',
    'num_transactions',
    'total_spent',
    'avg_spent',
    'last_purchase_date',
    'first_purchase_date',
    'avg_rating'
]

# Recency: days since last purchase
latest = agg['last_purchase_date'].max()
agg['recency_days'] = (latest - agg['last_purchase_date']).dt.days


In [14]:
last_prod = (
    df_trans.sort_values(['customer_id_legacy', 'purchase_date'])
            .groupby('customer_id_legacy')
            .tail(1)[['customer_id_legacy', 'product_category']]
)

agg = agg.merge(last_prod, on='customer_id_legacy', how='left')


## Fxing the ID Mismatch and merging
To enable the merge, I cleaned the customer_id_new column in the social dataset. Using Regular Expressions (regex), I extracted only the numeric digits from strings like 'A178' to create a matching integer key. I proceeded to perform a Left Join, keeping all social profiles and attaching transaction history where available. Missing transaction data (NaN) is filled with 0 or -1 to indicate 'no purchase history'

In [ ]:
df_social['customer_id_legacy'] = df_social['customer_id_new'].str.extract('(\d+)')
df_social['customer_id_legacy'] = df_social['customer_id_legacy'].astype(int)

In [17]:
df_merged = df_social.merge(agg, on='customer_id_legacy', how='left')

# Fill missing transaction features for customers with no history
df_merged['num_transactions'] = df_merged['num_transactions'].fillna(0)
df_merged['total_spent'] = df_merged['total_spent'].fillna(0)
df_merged['avg_spent'] = df_merged['avg_spent'].fillna(0)
df_merged['avg_rating'] = df_merged['avg_rating'].fillna(0)
df_merged['recency_days'] = df_merged['recency_days'].fillna(-1)

# product_category remains NaN for customers who never purchased (can drop or predict separately)


In [18]:
df_merged.head()
df_merged.info()
df_merged.isna().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   customer_id_new          155 non-null    object        
 1   social_media_platform    155 non-null    object        
 2   engagement_score         155 non-null    int64         
 3   purchase_interest_score  155 non-null    float64       
 4   review_sentiment         155 non-null    object        
 5   customer_id_legacy       155 non-null    int64         
 6   num_transactions         155 non-null    float64       
 7   total_spent              155 non-null    float64       
 8   avg_spent                155 non-null    float64       
 9   last_purchase_date       112 non-null    datetime64[ns]
 10  first_purchase_date      112 non-null    datetime64[ns]
 11  avg_rating               155 non-null    float64       
 12  recency_days             155 non-nul

,0
customer_id_new,0
social_media_platform,0
engagement_score,0
purchase_interest_score,0
review_sentiment,0
customer_id_legacy,0
num_transactions,0
total_spent,0
avg_spent,0
last_purchase_date,43


## Data Preparation for Modeling
I split the merged data into two sets:

Training Set: Customers who have a transaction history (and thus a known product_category).

Unlabeled Set: Customers with no history, whose preferred category we want to predict.

I separated the features (X) from my target (y). The target (product_category) is encoded into integers using LabelEncoder.

In [19]:
df_train = df_merged.dropna(subset=["product_category"]).copy()

# Rows with no purchase history (use later for prediction)
df_unlabeled = df_merged[df_merged["product_category"].isna()].copy()

df_train.shape, df_unlabeled.shape

((112, 14), (43, 14))

In [21]:
df = df_train.copy()

In [22]:
label_encoder = LabelEncoder()
df["product_category_encoded"] = label_encoder.fit_transform(df["product_category"])

In [31]:
X = df.drop(["product_category", "product_category_encoded", "last_purchase_date", "first_purchase_date", 'customer_id_new','customer_id_legacy',], axis=1)
y = df["product_category_encoded"]

In [32]:
categorical_cols = ["social_media_platform", "review_sentiment"]
numerical_cols = [col for col in X.columns if col not in categorical_cols and col != "customer_id_new"]


##Preprocessing Pipeline
I defined a ColumnTransformer to handle different data types automatically:

Categorical Features (social_media_platform, review_sentiment): These are transformed using OneHotEncoder to create binary variables for each category.

Numerical Features: These are passed through without change (passthrough).

This ensures the machine learning model receives purely numeric input.

In [33]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols)
    ],
    remainder="passthrough"
)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape

((89, 9), (23, 9))

## Random Forest Model
I constructed a pipeline that first applies the preprocessing steps and then feeds the data into a Random Forest Classifier. We set class_weight='balanced' to handle potential imbalances in product popularity. The initial results show an accuracy of roughly 65%, with some precision issues on under-represented classes

In [46]:
model = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("clf", RandomForestClassifier(
        n_estimators=400,
        max_depth= 15,
        random_state=42,
        class_weight="balanced"
    ))
])

In [47]:
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Metrics
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="weighted")

print("Accuracy:", acc)
print("F1 Score:", f1)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Accuracy: 0.6521739130434783
F1 Score: 0.6115942028985507

Classification Report:
              precision    recall  f1-score   support

       Books       0.25      0.25      0.25         4
    Clothing       1.00      0.50      0.67         4
 Electronics       0.67      1.00      0.80         6
   Groceries       0.00      0.00      0.00         2
      Sports       0.75      0.86      0.80         7

    accuracy                           0.65        23
   macro avg       0.53      0.52      0.50        23
weighted avg       0.62      0.65      0.61        23



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## XGBoost Model
I attempted to improve performance by using XGBoost. We use the same preprocessing pipeline. The model achieves 100% accuracy on the test set. While this looks perfect, given the small dataset size, this suggests the model may be overfitting (memorizing the data)

In [48]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [49]:
# Copy training dataset
df = df_train.copy()

# Encode target
label_encoder = LabelEncoder()
df["product_category_encoded"] = label_encoder.fit_transform(df["product_category"])

# Drop columns not used as features
drop_cols = ['customer_id_new', 'customer_id_legacy', 'last_purchase_date', 'first_purchase_date', 'product_category']
X = df.drop(columns=drop_cols)
y = df["product_category_encoded"]

# Identify categorical columns
categorical_cols = ['social_media_platform', 'review_sentiment']
numerical_cols = [col for col in X.columns if col not in categorical_cols]


In [50]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)


In [51]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [52]:
xgb_model = Pipeline([
    ('preprocess', preprocessor),
    ('clf', XGBClassifier(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.1,
        objective='multi:softprob',
        eval_metric='mlogloss',
        use_label_encoder=False,
        random_state=42
    ))
])

# Train model
xgb_model.fit(X_train, y_train)


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [07:20:27] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['social_media_platform',
                                                   'review_sentiment'])])),
                ('clf',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=Non...
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=6, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=300, n_jobs=None,
                               num_parallel_tree=None, ...))])

In [53]:
y_pred = xgb_model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", acc)
print("Weighted F1 Score:", f1)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Accuracy: 1.0
Weighted F1 Score: 1.0

Classification Report:
              precision    recall  f1-score   support

       Books       1.00      1.00      1.00         4
    Clothing       1.00      1.00      1.00         4
 Electronics       1.00      1.00      1.00         6
   Groceries       1.00      1.00      1.00         2
      Sports       1.00      1.00      1.00         7

    accuracy                           1.00        23
   macro avg       1.00      1.00      1.00        23
weighted avg       1.00      1.00      1.00        23



## Predicting on Unlabeled Data
I utilized the trained model to generate insights. I took the Unlabeled customers (those with no purchase history) and predict which product category they are most likely to be interested in based solely on their social media profile and engagement scores. These predictions are appended to the dataframe for marketing use.

In [42]:
X_new = df_unlabeled.drop(columns=['customer_id_new', 'customer_id_legacy',
                                   'last_purchase_date', 'first_purchase_date', 'product_category'])

# Predict
y_new_pred_encoded = model.predict(X_new)
y_new_pred = label_encoder.inverse_transform(y_new_pred_encoded)

# Add predictions back to df_unlabeled
df_unlabeled['predicted_product_category'] = y_new_pred
df_unlabeled[['customer_id_new', 'predicted_product_category']]

,customer_id_new,predicted_product_category
0,A178,Electronics
4,A197,Clothing
7,A196,Electronics
9,A178,Clothing
16,A109,Electronics
17,A155,Clothing
18,A116,Clothing
20,A116,Clothing
23,A168,Electronics
27,A165,Electronics


In [43]:
print("=== Variable Types ===")
print(df_merged.dtypes)

# Summary statistics for numeric features
print("\n=== Numeric Summary ===")
print(df_merged.describe())

# Summary for categorical features
print("\n=== Categorical Summary ===")
print(df_merged.describe(include='object'))


=== Variable Types ===
customer_id_new                    object
social_media_platform              object
engagement_score                    int64
purchase_interest_score           float64
review_sentiment                   object
customer_id_legacy                  int64
num_transactions                  float64
total_spent                       float64
avg_spent                         float64
last_purchase_date         datetime64[ns]
first_purchase_date        datetime64[ns]
avg_rating                        float64
recency_days                      float64
product_category                   object
dtype: object

=== Numeric Summary ===
       engagement_score  purchase_interest_score  customer_id_legacy  \
count        155.000000               155.000000          155.000000   
mean          75.606452                 3.152903          148.625806   
min           50.000000                 1.000000          100.000000   
25%           63.500000                 2.350000          123.

In [45]:
df_merged.to_csv("merged_customer_dataset.csv", index=False)

# If using Google Colab, you can download it locally
from google.colab import files
files.download("merged_customer_dataset.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>